In [3]:
from datetime import datetime
import sys
import traceback
import numpy as np
from datetime import datetime
import sys
import traceback
import scipy.io as sio
import torch

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import scipy.io as sio
from scipy.io import savemat
import numpy as np
import math
import random
import time
from collections import OrderedDict
import matplotlib.pyplot as plt
from itertools import cycle
import itertools as it
from pathlib import Path
import os
#import mysql.connector as mysql

In [15]:
import torch
import torch.nn as nn
from collections import OrderedDict


class ConvBN(nn.Sequential):
    def __init__(self, in_planes, out_planes, kernel_size, groups=1):
        if not isinstance(kernel_size, int):
            padding = [(i - 1) // 2 for i in kernel_size]
        else:
            padding = (kernel_size - 1) // 2
        super(ConvBN, self).__init__(OrderedDict([
            ('conv', nn.Conv2d(in_planes, out_planes, kernel_size, padding=padding, groups=groups)),
            ('bn', nn.BatchNorm2d(out_planes, eps=1e-03, momentum=0.99))
        ]))


class RefineNet(nn.Module):
    def __init__(self, img_channels=2):
        super(RefineNet, self).__init__()

        self.conv = nn.Sequential(OrderedDict([
            ("first_conv1x7", ConvBN(img_channels, 8, [1, 7])),
            ("LeakyReLU_1", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("second_conv1x7", ConvBN(8, 16, [1, 7])),
            ("LeakyReLU_2", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("third_conv1x7", ConvBN(16, 2, [1, 7])),
        ]))

        self.conv_1 = nn.Sequential(OrderedDict([
            ("first_conv1x5", ConvBN(img_channels, 8, [1, 5])),
            ("LeakyReLU_1", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("second_conv1x5", ConvBN(8, 16, [1, 5])),
            ("LeakyReLU_2", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("third_conv1x5", ConvBN(16, 2, [1, 5])),
        ]))

        self.conv1x1 = ConvBN(4, 2, [1, 7])
        self.relu = nn.LeakyReLU(negative_slope=0.3, inplace=True)

    def forward(self, x):
        ori_x = x.clone()
        x_1 = self.conv(x)
        x_2 = self.conv_1(x)
        x = torch.cat((x_1, x_2), dim=1)
        x = self.relu(x)
        x = self.conv1x1(x)
        return self.relu(x + ori_x)


class Encoder_Compression(nn.Module):
    def __init__(self):
        super(Encoder_Compression, self).__init__()
        self.conv = nn.Sequential(OrderedDict([
            ("conv1", ConvBN(64, 32, [1, 7])),
            ("LeakyReLU_1", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("conv2", ConvBN(32, 16, [1, 7])),
            ("LeakyReLU_2", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            #("third_Conv1x7",ConvBN(16,8,[1,7])),         for CR=8 
        ]))

        self.conv_2 = ConvBN(64, 16, [1, 7])
        self.conv_3 = ConvBN(32, 16, [1, 7])
        self.relu = nn.LeakyReLU(negative_slope=0.3, inplace=True)

    def forward(self, x):
        x_1 = self.conv(x)
        x_2 = self.conv_2(x)
        x = torch.cat((x_1, x_2), dim=1)
        x = self.relu(x)
        x = self.conv_3(x)
        return self.relu(x)


class CLLWCsiNet(nn.Module):
    def __init__(self, reduction=4, residual_num=2):
        super(CLLWCsiNet, self).__init__()
        self.encoder_p1 = nn.Sequential(OrderedDict([
            ("conv1x7", ConvBN(2, 2, [1, 7])),
            ("LeakyReLU_1", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("conv7x1", ConvBN(2, 2, [7, 1])),
            ("LeakyReLU_2", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ]))

        self.encoder_p2 = nn.Sequential(OrderedDict([
            ('conv1x5', ConvBN(2, 2, [1, 5])),
            ('LeakyReLU_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ('conv5x1', ConvBN(2, 2, [5, 1])),
        ]))

        self.encoder_p3 = nn.Sequential(OrderedDict([
            ('conv1x3', ConvBN(2, 2, [1, 3])),
            ('LeakyReLU_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ('conv3x1', ConvBN(2, 2, [3, 1])),
        ]))

        self.con1x1 = ConvBN(6, 2, [1, 7])
        self.relu = nn.LeakyReLU(negative_slope=0.3, inplace=True)

        self.encoder_compression = Encoder_Compression()

        self.decoder_get_feedback_in_UE = nn.Sequential(OrderedDict([
            ("conv1", ConvBN(16, 32, [1, 7])),
            ("LeakyReLU_1", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("conv2", ConvBN(32, 64, [1, 7])),
            ("LeakyReLU_2", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ]))

        self.remove_AGN = nn.Sequential(OrderedDict([
            ("conv1", ConvBN(16, 32, [1, 7])),
            ("LeakyReLU_1", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
            ("conv2", ConvBN(32, 64, [1, 7])),
            ("LeakyReLU_2", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ]))

        self.decoder_refine_net = nn.ModuleList([RefineNet(2) for _ in range(residual_num)])

        self._last_cov = nn.Sequential(OrderedDict([
            ("last_conv", ConvBN(2, 2, [1, 7])),
            ("activation", nn.Sigmoid())
        ]))

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    # def adding_noise(self, x):
    #     signal_power = torch.mean(x**2)
    #     SNR_dB = 40
    #     SNR_linear = 10**(SNR_dB / 10)
    #     noise_power = signal_power / SNR_linear
    #     noise = torch.randn_like(x) * torch.sqrt(noise_power)
    #    return x + noise

    def forward(self, x):
        batch_size = x.size(0)
        x_1 = self.encoder_p1(x)
        x_2 = self.encoder_p2(x)
        x_3 = self.encoder_p3(x)
        x = torch.cat((x_1, x_2, x_3), dim=1)
        x = self.relu(self.con1x1(x))
        x = x.view(batch_size, 64, 1, 32)

        x = self.encoder_compression(x)
        #x_noisy_feedback = self.adding_noise(x)
        #y = self.remove_AGN(x_noisy_feedback)
        x = self.decoder_get_feedback_in_UE(x)
        #x = self.relu(x - y)
        x = x.view(batch_size, 2, 32, 32)

        for refine_layer in self.decoder_refine_net:
            x = refine_layer(x)

        return self._last_cov(x)

In [16]:
model = CLLWCsiNet()

In [17]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [18]:
pytorch_total_params

70268

In [19]:
import torch
#from Trans_Net.utils import *


state_dict = torch.load(r'/kaggle/input/cllwcsinet-savemodels/CLLWCsiNet-Saved-models/cr-4/indoor/CLLWCsiNet_NoNoise_model_500_indoor_512_WithoutQuantization_pth',map_location=torch.device('cpu'))['model_state_dict']

/tmp/ipykernel_31/1553781525.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(r'/kaggle/input/cllwcsinet-savemodels/CLLWCsiNet-Saved-models/cr-4/i

In [20]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [21]:
model.eval()

CLLWCsiNet(
  (encoder_p1): Sequential(
    (conv1x7): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(1, 7), stride=(1, 1), padding=(0, 3))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    )
    (LeakyReLU_1): LeakyReLU(negative_slope=0.3, inplace=True)
    (conv7x1): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(7, 1), stride=(1, 1), padding=(3, 0))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    )
    (LeakyReLU_2): LeakyReLU(negative_slope=0.3, inplace=True)
  )
  (encoder_p2): Sequential(
    (conv1x5): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(1, 5), stride=(1, 1), padding=(0, 2))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    )
    (LeakyReLU_1): LeakyReLU(negative_slope=0.3, inplace=True)
    (conv5x1): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
      (bn): BatchNorm2d(2, eps=0.001, mom

In [22]:
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')  # Set quantization config
quantized_model = torch.quantization.quantize_dynamic(
    model, {nn.Linear}, dtype=torch.qint8  # Adjust the module types as per your model
)

/usr/local/lib/python3.11/dist-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [23]:
# For simplicity, I'm using a dummy input for calibration
dummy_input = torch.randn(1, 2, 32, 32)  # Modify according to your input shape
quantized_model(dummy_input)

tensor([[[[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.6015e-03,
           9.1594e-01, 6.5238e-01],
          [9.1728e-01, 9.9892e-01, 4.7074e-04,  ..., 1.3163e-01,
           6.8144e-01, 7.6978e-01],
          [1.0000e+00, 9.9997e-01, 1.0000e+00,  ..., 3.6751e-02,
           8.7335e-01, 4.1928e-01],
          ...,
          [1.0000e+00, 1.1364e-05, 6.3566e-01,  ..., 2.8783e-08,
           9.9183e-01, 9.8027e-01],
          [9.9841e-01, 9.9682e-01, 3.2049e-02,  ..., 1.1809e-05,
           9.8508e-01, 2.9185e-01],
          [1.0000e+00, 9.9980e-01, 9.9957e-01,  ..., 1.4717e-03,
           9.9368e-01, 2.0414e-01]],

         [[1.6770e-02, 9.1624e-01, 7.8061e-01,  ..., 9.7689e-01,
           3.8514e-01, 8.7960e-01],
          [8.8834e-01, 9.9677e-01, 4.4352e-05,  ..., 9.8468e-01,
           6.2998e-01, 6.9593e-01],
          [4.2592e-03, 9.9528e-01, 4.4093e-01,  ..., 9.9919e-01,
           9.2032e-01, 7.2109e-01],
          ...,
          [9.9264e-01, 8.0115e-01, 9.8544e-03,  ..., 1.0000

In [24]:
quantized_model.to('cpu')

CLLWCsiNet(
  (encoder_p1): Sequential(
    (conv1x7): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(1, 7), stride=(1, 1), padding=(0, 3))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    )
    (LeakyReLU_1): LeakyReLU(negative_slope=0.3, inplace=True)
    (conv7x1): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(7, 1), stride=(1, 1), padding=(3, 0))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    )
    (LeakyReLU_2): LeakyReLU(negative_slope=0.3, inplace=True)
  )
  (encoder_p2): Sequential(
    (conv1x5): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(1, 5), stride=(1, 1), padding=(0, 2))
      (bn): BatchNorm2d(2, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    )
    (LeakyReLU_1): LeakyReLU(negative_slope=0.3, inplace=True)
    (conv5x1): ConvBN(
      (conv): Conv2d(2, 2, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
      (bn): BatchNorm2d(2, eps=0.001, mom

In [25]:
criterion = nn.MSELoss().to('cpu')

In [26]:
envir = 'indoor'  # 'indoor' or 'outdoor'
# image params
img_height = 32
img_width = 32
img_channels = 2
img_total = img_height * img_width * img_channels
# network params
#residual_num = 2
encoded_dim = 512  # compress rate=1/4->dim.=512, compress rate=1/16->dim.=128, compress rate=1/32->dim.=64, compress rate=1/64->dim.=32

In [28]:
# envir = 'indoor'  # 'indoor' or 'outdoor'
# # image params
# img_height = 32
# img_width = 32
# img_channels = 2
# img_total = img_height * img_width * img_channels
# # network params
# residual_num = 2

# encoded_dim = 512  # compress rate=1/4->dim.=512, compress rate=1/16->dim.=128, compress rate=1/32->dim.=64, compress rate=1/64->dim.=32
# Data loading
if envir == 'indoor':
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Htrainin.mat')
    x_train = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Hvalin.mat')
    x_val = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Htestin.mat')
    x_test = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_HtestFin_all.mat')
    X_test = mat['HF_all']  # array

elif envir == 'outdoor':
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Htrainout.mat')
    x_train = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Hvalout.mat')
    x_val = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Htestout.mat')
    x_test = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_HtestFout_all.mat')
    X_test = mat['HF_all']  # array

In [29]:
import numpy as np

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

x_train_length = len(x_train)
x_val_length = len(x_val)
x_test_length = len(x_test)

x_train = np.reshape(x_train, (x_train_length, img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format
x_val = np.reshape(x_val, (x_val_length, img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (x_test_length, img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format

x_train = torch.tensor(x_train)
x_val = torch.tensor(x_val)
x_test = torch.tensor(x_test)

In [30]:
x_test.shape

torch.Size([20000, 2, 32, 32])

In [31]:
img_height = 32
img_width = 32
img_channels = 2

In [32]:
import math
with torch.no_grad():

    #torch.cuda.empty_cache()
    quantized_model.eval()
    device='cpu'
    x_hat = quantized_model(x_test)
    #torch.quantization.convert(quantized_model, inplace=True)
    x_test = x_test.to('cpu')
    x_hat=x_hat.to('cpu')

    # Calcaulating the NMSE and rho
    # if envir == 'indoor':
    #     mat = sio.loadmat('D:\Cost2100\DATA_HtestFin_all.mat')
    #     X_test = mat['HF_all']  # array

    # elif envir == 'outdoor':
    #     mat = sio.loadmat('D:\Cost2100\DATA_HtestFout_all.mat')
    #     X_test = mat['HF_all']  # array

    #X_test = torch.tensor(X_test)
    #X_test = torch.reshape(X_test, (len(X_test), img_height, 125))
    x_test_real = torch.reshape(x_test[:, 0, :, :], (len(x_test), -1))
    x_test_imag = torch.reshape(x_test[:, 1, :, :], (len(x_test), -1))
    x_test_C = x_test_real - 0.5 + 1j * (x_test_imag - 0.5)
    x_hat_real = torch.reshape(x_hat[:, 0, :, :], (len(x_hat), -1))
    x_hat_imag = torch.reshape(x_hat[:, 1, :, :], (len(x_hat), -1))
    x_hat_C = x_hat_real - 0.5 + 1j * (x_hat_imag - 0.5)
    x_hat_F = torch.reshape(x_hat_C, (len(x_hat_C), img_height, img_width))
    X_hat = torch.fft.fft(torch.cat((x_hat_F, torch.zeros((len(x_hat_C), img_height, 257 - img_width))), axis=2), axis=2)
    X_hat = X_hat[:, :, 0:125]

    #n1 = torch.sqrt(torch.sum(torch.conj(X_test) * X_test, axis=1))
    #n2 = torch.sqrt(torch.sum(torch.conj(X_hat) * X_hat, axis=1))
    #aa = abs(torch.sum(torch.conj(X_test) * X_hat, axis=1))
    #rho = torch.mean(aa / (n1 * n2), axis=1)
    X_hat = torch.reshape(X_hat, (len(X_hat), -1))
    #X_test = torch.reshape(X_test, (len(X_test), -1))
    power = torch.sum(abs(x_test_C) ** 2, axis=1)
    power_d = torch.sum(abs(X_hat) ** 2, axis=1)
    mse = torch.sum(abs(x_test_C - x_hat_C) ** 2, axis=1)
    NMSE = 10 * math.log10(torch.mean(mse / power))
    #Correlation = torch.mean(rho).item().real

    # print("In " + envir + " environment")
    # print("When dimension is", encoded_dim)
    print("NMSE is ", NMSE)
    #print("Correlation is ", Correlation)
#
# file = 'CsiNet_' + (envir) + '_dim' + str(encoded_dim) + time.strftime('_%m_%d_%H_%M')
# outfile = "result/result_%s.mat" % file
# savemat(outfile, {'train_loss_history': train_loss_history,
#                   'val_loss_history': val_loss_history,
#                   'training_time': training_time,
#                   'NMSE': NMSE,
#                   'Correlation': Correlation})

NMSE is  -14.422470030227604


In [35]:
# ====== COMPREHENSIVE QUANTIZED MODEL BENCHMARK ======
import time
import torch
from tabulate import tabulate

def benchmark_model(model, input_shape=(2, 32, 32), warmup=10, repetitions=100, batch_sizes=[1, 8, 16, 32, 64], quantized=False):
    """Run comprehensive benchmarks for both quantized and regular models"""
    results = []
    model.eval()
    
    # Prepare model
    if quantized:
        model = torch.quantization.quantize_dynamic(
            model, 
            {torch.nn.Conv2d, torch.nn.Linear},  # Quantize Conv2d layers too
            dtype=torch.qint8
        )
        model_type = "Quantized"
    else:
        model_type = "FP32"
    
    # Benchmark for CPU and GPU (if available and not quantized)
    devices = ['cpu']
    if torch.cuda.is_available() and not quantized:
        devices.append('cuda')
    
    for device in devices:
        # Move model to device
        model.to(device)
        
        # Single sample benchmark
        dummy_input = torch.randn(1, *input_shape).to(device)
        
        # Warm-up
        with torch.no_grad():
            for _ in range(warmup):
                _ = model(dummy_input)
            if device == 'cuda':
                torch.cuda.synchronize()
        
        # Measurement
        start_time = time.time()
        with torch.no_grad():
            for _ in range(repetitions):
                _ = model(dummy_input)
                if device == 'cuda':
                    torch.cuda.synchronize()
        avg_time = (time.time() - start_time) * 1000 / repetitions
        
        results.append([
            f"{device.upper()} ({model_type})",
            "Single",
            f"{avg_time:.2f} ms",
            f"{1000/avg_time:.2f} samples/sec"
        ])
        
        # Batch processing benchmark
        for bs in batch_sizes:
            dummy_batch = torch.randn(bs, *input_shape).to(device)
            
            # Warm-up
            with torch.no_grad():
                _ = model(dummy_batch)
                if device == 'cuda':
                    torch.cuda.synchronize()
            
            # Measure
            start = time.time()
            with torch.no_grad():
                _ = model(dummy_batch)
                if device == 'cuda':
                    torch.cuda.synchronize()
            
            total_time = (time.time() - start) * 1000
            time_per_sample = total_time / bs
            
            results.append([
                f"{device.upper()} ({model_type})",
                f"Batch {bs}",
                f"{total_time:.2f} ms",
                f"{1000/time_per_sample:.2f} samples/sec"
            ])
    
    return results

# Run benchmarks
print("\n=== Model Benchmark Results ===")
benchmark_results = benchmark_model(model, quantized=False)
quantized_results = benchmark_model(model, quantized=True)

# Combine and print results
all_results = benchmark_results + quantized_results
headers = ["Configuration", "Test Type", "Inference Time", "Throughput"]
print(tabulate(all_results, headers=headers))

# Additional metrics
print("\n=== Quantization Benefits ===")
# Find matching CPU results to compare
fp32_cpu_single = next(r for r in benchmark_results if "CPU (FP32)" in r[0] and "Single" in r[1])
quant_cpu_single = next(r for r in quantized_results if "CPU (Quantized)" in r[0] and "Single" in r[1])

speedup = float(fp32_cpu_single[2].split()[0]) / float(quant_cpu_single[2].split()[0])
print(f"Quantization speedup (CPU single sample): {speedup:.2f}x")

# Model size comparison
def get_model_size(model):
    torch.save(model.state_dict(), "temp.p")
    size = os.path.getsize("temp.p")/1e6  # in MB
    os.remove("temp.p")
    return size

fp32_size = get_model_size(model)
quantized_model = torch.quantization.quantize_dynamic(model, {torch.nn.Conv2d}, dtype=torch.qint8)
quant_size = get_model_size(quantized_model)

print(f"\nModel size reduction: {fp32_size:.2f}MB -> {quant_size:.2f}MB ({fp32_size/quant_size:.1f}x smaller)")


=== Model Benchmark Results ===
Configuration    Test Type    Inference Time    Throughput
---------------  -----------  ----------------  --------------------
CPU (FP32)       Single       2.92 ms           342.03 samples/sec
CPU (FP32)       Batch 1      2.99 ms           334.39 samples/sec
CPU (FP32)       Batch 8      8.78 ms           911.04 samples/sec
CPU (FP32)       Batch 16     11.16 ms          1433.95 samples/sec
CPU (FP32)       Batch 32     17.08 ms          1873.50 samples/sec
CPU (FP32)       Batch 64     43.58 ms          1468.58 samples/sec
CUDA (FP32)      Single       3.27 ms           305.45 samples/sec
CUDA (FP32)      Batch 1      3.25 ms           308.16 samples/sec
CUDA (FP32)      Batch 8      3.43 ms           2335.20 samples/sec
CUDA (FP32)      Batch 16     3.32 ms           4814.47 samples/sec
CUDA (FP32)      Batch 32     3.31 ms           9674.05 samples/sec
CUDA (FP32)      Batch 64     3.90 ms           16407.03 samples/sec
CPU (Quantized)  Single    